# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [34]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn import tree
from sklearn.model_selection import train_test_split

from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression


%matplotlib inline

In [35]:
df = pd.read_csv('previsao_de_renda.csv')

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [37]:
df_numerico = df.select_dtypes(include=['number'])

df_numerico.head()

,Unnamed: 0,id_cliente,qtd_filhos,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,15056,0,26,6.602740,1.0,8060.34
1,1,9968,0,28,7.183562,2.0,1852.15
2,2,4312,0,35,0.838356,2.0,2253.89
3,3,10639,1,30,4.846575,3.0,6600.77
4,4,7064,0,33,4.293151,1.0,6475.97


In [38]:
X = df_numerico.drop(columns=['renda'])
y = df_numerico['renda'] 

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [40]:
modelo = DecisionTreeRegressor(max_depth=5, random_state=30)
modelo.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5, random_state=30)

In [41]:
X = df.drop(columns=['renda'])  
y = df['renda']                 

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

In [42]:
numericas = X_train.select_dtypes(include=['int64', 'float64', 'bool']).columns.tolist()
categoricas = X_train.select_dtypes(include='object').columns.tolist()

preprocessador = ColumnTransformer(transformers=[
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numericas),
    
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), categoricas)
])

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
resultados = {}

for alpha in alphas:
    modelo = Pipeline(steps=[
        ('preprocessador', preprocessador),
        ('regressor', Ridge(alpha=alpha))
    ])
    
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    resultados[alpha] = r2

for alpha, r2 in resultados.items():
    print(f"Alpha: {alpha:.3f} | R² na base de teste: {r2:.4f}")


Alpha: 0.000 | R² na base de teste: 0.2672
Alpha: 0.001 | R² na base de teste: 0.2680
Alpha: 0.005 | R² na base de teste: 0.2680
Alpha: 0.010 | R² na base de teste: 0.2680
Alpha: 0.050 | R² na base de teste: 0.2680
Alpha: 0.100 | R² na base de teste: 0.2680


In [43]:
from sklearn.linear_model import LinearRegression

modelo_lr = Pipeline(steps=[
    ('preprocessador', preprocessador),
    ('regressor', LinearRegression())
])

modelo_lr.fit(X_train, y_train)
y_pred_lr = modelo_lr.predict(X_test)
r2_lr = r2_score(y_test, y_pred_lr)
print(f"Linear Regression (sem regularização) | R²: {r2_lr:.4f}")



Linear Regression (sem regularização) | R²: 0.2680


In [44]:
def stepwise_selection(X, y, threshold_in=0.01, verbose=True):
    included = []
    X_dummies = pd.get_dummies(X, drop_first=True)  # converte todas as categorias para números
    
    while True:
        changed = False
        excluded = list(set(X_dummies.columns) - set(included))
        new_pval = pd.Series(index=excluded, dtype=float)
        
        for new_column in excluded:
            try:
                model = sm.OLS(y, sm.add_constant(X_dummies[included + [new_column]])).fit()
                new_pval[new_column] = model.pvalues[new_column]
            except:
                continue
        
        if new_pval.empty:
            break
        
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f"Adicionado: {best_feature}, p-valor = {best_pval:.6f}")
        
        if not changed:
            break

    return included

X_treino_limpo = X_train.dropna()
y_treino_limpo = y_train.loc[X_treino_limpo.index]

variaveis_selecionadas = stepwise_selection(X_treino_limpo, y_treino_limpo)

X_dummies = pd.get_dummies(X_treino_limpo[variaveis_selecionadas], drop_first=True)
X_final = sm.add_constant(X_dummies)
modelo = sm.OLS(y_treino_limpo, X_final).fit()

print(modelo.summary())


Adicionado: tempo_emprego, p-valor = 0.000000
Adicionado: qt_pessoas_residencia, p-valor = 0.000004
Adicionado: qtd_filhos, p-valor = 0.000398
                            OLS Regression Results                            
Dep. Variable:                  renda   R-squared:                       0.151
Model:                            OLS   Adj. R-squared:                  0.151
Method:                 Least Squares   F-statistic:                     552.6
Date:                Thu, 08 May 2025   Prob (F-statistic):               0.00
Time:                        10:49:49   Log-Likelihood:                -97159.
No. Observations:                9313   AIC:                         1.943e+05
Df Residuals:                    9309   BIC:                         1.944e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|  

Utilizar o LASSO e o modelo de regressão Linear nos deram melhores R2 com essa base da dados e tratativas, obtivemos quase metade com o stpwise.

Por ter sido obtido o mesmo R2 que LASSO e achar mais simples eu preferiria seguir com o metódo ridge, por mais mais direto.

In [45]:
X = df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente', 'renda'])
y = df['renda']

X_dummies = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [46]:
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print(f" Random Forest R²: {r2_score(y_test, y_pred_rf):.4f}")

 Random Forest R²: 0.4112


In [47]:
tree = DecisionTreeRegressor(max_depth=6, random_state=42)
tree.fit(X_train, y_train)

y_pred_tree = tree.predict(X_test)

r2_tree = r2_score(y_test, y_pred_tree)
print(f"Árvore de Regressão R² na base de teste: {r2_tree:.4f}")

Árvore de Regressão R² na base de teste: 0.3797


Ao ajustar a arvore de regressão obtivemos sim um melhor R2, comparando as trativas normais impostas no exercicio. Mas o melhor R2 veio ao utilizarmos o RandomForestRegressor, algo que nos ajudou a ter um R2 muito interessante para nossa base de dados.